# 📅 Analyse EDA - Patterns Temporels

Ce notebook se concentre sur l'analyse temporelle des données d'entraînement.

## Objectifs
- Analyser l'évolution des performances dans le temps
- Identifier les patterns et tendances
- Détecter les périodes de progression/stagnation
- Analyser la régularité d'entraînement
- Préparer les features temporelles pour le ML

## 🔧 Imports et configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (14, 8)

print("📅 Notebook d'analyse temporelle initialisé")

## 📁 Chargement et préparation des données

In [ ]:
# Chargement des données
df_raw = pd.read_csv('../examples/sample_data.csv')

# Nettoyage et préparation
df = df_raw.copy()
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Poids_kg'] = df['Poids / Distance'].str.replace(' kg', '').str.replace(',', '.').astype(float)
df['Reps'] = df['Répétitions / Temps'].str.extract(r'(\d+)').astype(float)
df['Volume'] = df['Poids_kg'] * df['Reps']
df['Type_serie'] = df['Série / Série d\'échauffement / Série de récupération']
df['Sautee'] = df['Sautée'].map({'Oui': True, 'Non': False})

# Ajout d'informations temporelles
df['Jour_Semaine'] = df['Date'].dt.day_name()
df['Semaine'] = df['Date'].dt.isocalendar().week
df['Mois'] = df['Date'].dt.month
df['Jour_Numero'] = df['Date'].dt.dayofweek  # 0=Lundi, 6=Dimanche

# Tri par date pour l'analyse temporelle
df = df.sort_values('Date')

print(f"📊 Données préparées: {len(df)} sets")
print(f"📅 Période: du {df['Date'].min().strftime('%d/%m/%Y')} au {df['Date'].max().strftime('%d/%m/%Y')}")
print(f"⏱️ Durée totale: {(df['Date'].max() - df['Date'].min()).days} jours")

## 📈 Évolution globale du volume

In [ ]:
# Analyse du volume par jour
print("📈 ÉVOLUTION DU VOLUME D'ENTRAÎNEMENT")
print("=" * 50)

daily_stats = df.groupby('Date').agg({
    'Volume': ['sum', 'count', 'mean'],
    'Poids_kg': 'mean',
    'Reps': 'mean',
    'Entraînement': 'first',
    'Région': 'nunique'
}).round(2)

daily_stats.columns = ['Volume_Total', 'Nb_Sets', 'Volume_Moyen_Set', 'Poids_Moyen', 'Reps_Moyen', 'Type_Entrainement', 'Nb_Regions']
daily_stats = daily_stats.reset_index()

print("📋 Statistiques quotidiennes:")
print(daily_stats)

# Calcul des tendances
daily_stats['Jour_Index'] = range(len(daily_stats))
slope_volume, intercept_volume, r_value_volume, p_value_volume, std_err_volume = stats.linregress(
    daily_stats['Jour_Index'], daily_stats['Volume_Total']
)

print(f"\n📊 TENDANCE GÉNÉRALE:")
print(f"   Pente du volume: {slope_volume:.2f} kg/jour")
print(f"   Corrélation: {r_value_volume:.3f}")
print(f"   Tendance: {'📈 Croissante' if slope_volume > 0 else '📉 Décroissante' if slope_volume < 0 else '➡️ Stable'}")
print(f"   Significativité: {'✅ Significative' if p_value_volume < 0.05 else '⚠️ Non significative'} (p={p_value_volume:.3f})")

In [ ]:
# Visualisation de l'évolution du volume
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📈 Évolution Temporelle des Métriques d\'Entraînement', fontsize=16, fontweight='bold')

# 1. Volume total par jour
axes[0,0].plot(daily_stats['Date'], daily_stats['Volume_Total'], 
               marker='o', linewidth=2, markersize=8, color='navy')
# Ligne de tendance
trend_line = slope_volume * daily_stats['Jour_Index'] + intercept_volume
axes[0,0].plot(daily_stats['Date'], trend_line, '--', color='red', alpha=0.7, 
               label=f'Tendance: {slope_volume:.1f} kg/jour')
axes[0,0].set_title('Volume Total par Jour')
axes[0,0].set_ylabel('Volume (kg)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Nombre de sets par jour
axes[0,1].bar(daily_stats['Date'], daily_stats['Nb_Sets'], color='steelblue', alpha=0.7)
axes[0,1].set_title('Nombre de Sets par Jour')
axes[0,1].set_ylabel('Nombre de Sets')
axes[0,1].grid(True, alpha=0.3)

# 3. Poids moyen par jour
axes[1,0].plot(daily_stats['Date'], daily_stats['Poids_Moyen'], 
               marker='s', linewidth=2, markersize=6, color='darkgreen')
axes[1,0].set_title('Poids Moyen par Jour')
axes[1,0].set_ylabel('Poids Moyen (kg)')
axes[1,0].grid(True, alpha=0.3)

# 4. Volume moyen par set
axes[1,1].plot(daily_stats['Date'], daily_stats['Volume_Moyen_Set'], 
               marker='^', linewidth=2, markersize=6, color='purple')
axes[1,1].set_title('Volume Moyen par Set')
axes[1,1].set_ylabel('Volume/Set (kg)')
axes[1,1].grid(True, alpha=0.3)

# Formatage des axes de dates
for ax in axes.flat:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🏋️ Progression par exercice

In [ ]:
# Analyse de la progression par exercice
print("🏋️ PROGRESSION PAR EXERCICE")
print("=" * 50)

# Pour chaque exercice, analyser l'évolution du poids et volume
exercises = df['Exercice'].unique()
progression_summary = []

for exercise in exercises:
    exercise_data = df[df['Exercice'] == exercise].copy()
    exercise_data = exercise_data.sort_values('Date')
    
    if len(exercise_data) >= 2:  # Au moins 2 points pour calculer une progression
        # Calcul de la progression du poids max
        first_weight = exercise_data['Poids_kg'].iloc[0]
        last_weight = exercise_data['Poids_kg'].iloc[-1]
        max_weight = exercise_data['Poids_kg'].max()
        
        # Calcul de la progression du volume
        first_volume = exercise_data['Volume'].iloc[0]
        last_volume = exercise_data['Volume'].iloc[-1]
        avg_volume = exercise_data['Volume'].mean()
        
        # Régression linéaire pour la tendance
        days = [(date - exercise_data['Date'].iloc[0]).days for date in exercise_data['Date']]
        if len(set(exercise_data['Poids_kg'])) > 1:  # Variation de poids
            slope_weight, _, r_weight, p_weight, _ = stats.linregress(days, exercise_data['Poids_kg'])
        else:
            slope_weight, r_weight, p_weight = 0, 0, 1
        
        progression_summary.append({
            'Exercice': exercise,
            'Nb_Sessions': len(exercise_data),
            'Premier_Poids': first_weight,
            'Dernier_Poids': last_weight,
            'Poids_Max': max_weight,
            'Progression_Poids': last_weight - first_weight,
            'Progression_Pct': ((last_weight - first_weight) / first_weight * 100) if first_weight > 0 else 0,
            'Tendance_Poids_Jour': slope_weight,
            'Correlation_Temps': r_weight,
            'Volume_Moyen': avg_volume,
            'Progression_Volume': last_volume - first_volume
        })

progression_df = pd.DataFrame(progression_summary)
progression_df = progression_df.round(2).sort_values('Progression_Pct', ascending=False)

print("📊 Résumé des progressions:")
print(progression_df)

# Identification des meilleures et moins bonnes progressions
best_progression = progression_df.iloc[0]
worst_progression = progression_df.iloc[-1]

print(f"\n🏆 MEILLEURE PROGRESSION:")
print(f"   Exercice: {best_progression['Exercice']}")
print(f"   Progression: +{best_progression['Progression_Poids']:.1f}kg ({best_progression['Progression_Pct']:.1f}%)")
print(f"   Tendance: {best_progression['Tendance_Poids_Jour']:.3f} kg/jour")

print(f"\n📉 PROGRESSION À AMÉLIORER:")
print(f"   Exercice: {worst_progression['Exercice']}")
print(f"   Progression: {worst_progression['Progression_Poids']:.1f}kg ({worst_progression['Progression_Pct']:.1f}%)")
print(f"   Tendance: {worst_progression['Tendance_Poids_Jour']:.3f} kg/jour")

In [ ]:
# Visualisation de la progression par exercice
fig, axes = plt.subplots(len(exercises), 1, figsize=(14, 6*len(exercises)))
if len(exercises) == 1:
    axes = [axes]

fig.suptitle('🏋️ Progression Détaillée par Exercice', fontsize=16, fontweight='bold')

for i, exercise in enumerate(exercises):
    exercise_data = df[df['Exercice'] == exercise].sort_values('Date')
    
    # Graphique principal : poids dans le temps
    ax = axes[i]
    scatter = ax.scatter(exercise_data['Date'], exercise_data['Poids_kg'], 
                        c=exercise_data['Volume'], cmap='viridis', 
                        s=exercise_data['Reps']*10, alpha=0.7)
    
    # Ligne de tendance si données suffisantes
    if len(exercise_data) >= 2:
        z = np.polyfit(range(len(exercise_data)), exercise_data['Poids_kg'], 1)
        p = np.poly1d(z)
        ax.plot(exercise_data['Date'], p(range(len(exercise_data))), "--", 
               color='red', alpha=0.8, linewidth=2)
    
    ax.set_title(f'{exercise} - Évolution du Poids (couleur=volume, taille=reps)')
    ax.set_ylabel('Poids (kg)')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    # Colorbar pour le volume
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Volume (kg)')

plt.tight_layout()
plt.show()

## 📊 Analyse des patterns hebdomadaires

In [ ]:
# Analyse des patterns hebdomadaires
print("📊 ANALYSE DES PATTERNS HEBDOMADAIRES")
print("=" * 50)

# Répartition par jour de la semaine
weekly_patterns = df.groupby(['Jour_Semaine', 'Jour_Numero']).agg({
    'Volume': ['sum', 'count', 'mean'],
    'Date': 'nunique'
}).round(2)

weekly_patterns.columns = ['Volume_Total', 'Nb_Sets', 'Volume_Moyen', 'Nb_Jours_Entrainement']
weekly_patterns = weekly_patterns.reset_index().sort_values('Jour_Numero')

print("📅 Activité par jour de la semaine:")
print(weekly_patterns)

# Calcul de l'intensité par jour (volume/nombre de jours d'entraînement)
weekly_patterns['Intensite_Jour'] = weekly_patterns['Volume_Total'] / weekly_patterns['Nb_Jours_Entrainement']

print(f"\n🏆 Jour le plus actif: {weekly_patterns.loc[weekly_patterns['Volume_Total'].idxmax(), 'Jour_Semaine']}")
print(f"😴 Jour le moins actif: {weekly_patterns.loc[weekly_patterns['Volume_Total'].idxmin(), 'Jour_Semaine']}")
print(f"💪 Jour le plus intensif: {weekly_patterns.loc[weekly_patterns['Intensite_Jour'].idxmax(), 'Jour_Semaine']}")

In [ ]:
# Visualisation des patterns hebdomadaires
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('📅 Patterns Hebdomadaires d\'Entraînement', fontsize=16, fontweight='bold')

# 1. Volume total par jour de la semaine
axes[0,0].bar(weekly_patterns['Jour_Semaine'], weekly_patterns['Volume_Total'], 
              color='skyblue', alpha=0.8)
axes[0,0].set_title('Volume Total par Jour de la Semaine')
axes[0,0].set_ylabel('Volume Total (kg)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# 2. Nombre de sets par jour
axes[0,1].bar(weekly_patterns['Jour_Semaine'], weekly_patterns['Nb_Sets'], 
              color='lightgreen', alpha=0.8)
axes[0,1].set_title('Nombre de Sets par Jour de la Semaine')
axes[0,1].set_ylabel('Nombre de Sets')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

# 3. Intensité (volume/jour d'entraînement)
axes[1,0].bar(weekly_patterns['Jour_Semaine'], weekly_patterns['Intensite_Jour'], 
              color='orange', alpha=0.8)
axes[1,0].set_title('Intensité par Jour (Volume/Jour d\'entraînement)')
axes[1,0].set_ylabel('Intensité (kg)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# 4. Heatmap des régions par jour de la semaine
region_day_pivot = df.groupby(['Jour_Semaine', 'Région'])['Volume'].sum().unstack(fill_value=0)
sns.heatmap(region_day_pivot.T, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[1,1])
axes[1,1].set_title('Volume par Région et Jour (Heatmap)')
axes[1,1].set_xlabel('Jour de la Semaine')
axes[1,1].set_ylabel('Région Musculaire')

plt.tight_layout()
plt.show()

## ⏱️ Analyse de la régularité d'entraînement

In [ ]:
# Analyse de la régularité
print("⏱️ ANALYSE DE LA RÉGULARITÉ D'ENTRAÎNEMENT")
print("=" * 60)

# Calcul des intervalles entre sessions
training_dates = df['Date'].unique()
training_dates.sort()

intervals = []
for i in range(1, len(training_dates)):
    interval = (training_dates[i] - training_dates[i-1]).days
    intervals.append(interval)

if intervals:
    intervals_df = pd.DataFrame({
        'Date_Debut': training_dates[:-1],
        'Date_Fin': training_dates[1:],
        'Intervalle_Jours': intervals
    })
    
    print("📊 Statistiques des intervalles entre sessions:")
    print(f"   Intervalle moyen: {np.mean(intervals):.1f} jours")
    print(f"   Intervalle médian: {np.median(intervals):.1f} jours")
    print(f"   Écart-type: {np.std(intervals):.1f} jours")
    print(f"   Intervalle min: {min(intervals)} jours")
    print(f"   Intervalle max: {max(intervals)} jours")
    
    # Classification de la régularité
    if np.std(intervals) <= 1:
        regularity = "🎯 Très régulière"
    elif np.std(intervals) <= 2:
        regularity = "✅ Régulière"
    elif np.std(intervals) <= 3:
        regularity = "⚠️ Modérément irrégulière"
    else:
        regularity = "❌ Irrégulière"
    
    print(f"\n⏱️ Évaluation de la régularité: {regularity}")
    
    print("\n📅 Détail des intervalles:")
    print(intervals_df)
else:
    print("⚠️ Pas assez de données pour analyser la régularité")

In [ ]:
# Visualisation de la régularité
if intervals:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Histogramme des intervalles
    axes[0].hist(intervals, bins=max(1, len(set(intervals))), alpha=0.7, color='steelblue', edgecolor='black')
    axes[0].axvline(np.mean(intervals), color='red', linestyle='--', 
                   label=f'Moyenne: {np.mean(intervals):.1f} jours')
    axes[0].set_title('📊 Distribution des Intervalles entre Sessions')
    axes[0].set_xlabel('Intervalle (jours)')
    axes[0].set_ylabel('Fréquence')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 2. Évolution des intervalles dans le temps
    axes[1].plot(range(len(intervals)), intervals, marker='o', linewidth=2, markersize=6)
    axes[1].axhline(np.mean(intervals), color='red', linestyle='--', alpha=0.7, 
                   label=f'Moyenne: {np.mean(intervals):.1f} jours')
    axes[1].set_title('⏱️ Évolution des Intervalles dans le Temps')
    axes[1].set_xlabel('Numéro de la Pause')
    axes[1].set_ylabel('Intervalle (jours)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Pas assez de données pour visualiser la régularité")

## 🎲 Calcul des features temporelles pour ML

In [ ]:
# Calcul des features temporelles avancées
print("🎲 CALCUL DES FEATURES TEMPORELLES POUR ML")
print("=" * 60)

# Création d'un DataFrame enrichi avec des features temporelles
df_features = df.copy()

# 1. Features basées sur l'ordre chronologique
df_features = df_features.sort_values(['Exercice', 'Date'])
df_features['Session_Number'] = df_features.groupby('Exercice').cumcount() + 1

# 2. Rolling windows pour tendances
df_features['Volume_Rolling_3'] = df_features.groupby('Exercice')['Volume'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
df_features['Poids_Rolling_3'] = df_features.groupby('Exercice')['Poids_kg'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)

# 3. Features de progression
df_features['Poids_Progression'] = df_features.groupby('Exercice')['Poids_kg'].diff()
df_features['Volume_Progression'] = df_features.groupby('Exercice')['Volume'].diff()

# 4. Features relatives au maximum personnel
df_features['Poids_Max_Personnel'] = df_features.groupby('Exercice')['Poids_kg'].cummax()
df_features['Pct_Max_Personnel'] = (df_features['Poids_kg'] / df_features['Poids_Max_Personnel'] * 100).round(1)

# 5. Features temporelles cycliques
df_features['Jour_Semaine_Sin'] = np.sin(2 * np.pi * df_features['Jour_Numero'] / 7)
df_features['Jour_Semaine_Cos'] = np.cos(2 * np.pi * df_features['Jour_Numero'] / 7)

# 6. Délai depuis la dernière session de l'exercice
df_features['Jours_Depuis_Dernier'] = df_features.groupby('Exercice')['Date'].diff().dt.days

# 7. Volume cumulé
df_features['Volume_Cumule'] = df_features.groupby('Exercice')['Volume'].cumsum()

print("✅ Features temporelles calculées:")
new_features = ['Session_Number', 'Volume_Rolling_3', 'Poids_Rolling_3', 
                'Poids_Progression', 'Volume_Progression', 'Poids_Max_Personnel',
                'Pct_Max_Personnel', 'Jour_Semaine_Sin', 'Jour_Semaine_Cos',
                'Jours_Depuis_Dernier', 'Volume_Cumule']

for feature in new_features:
    print(f"   • {feature}")

# Aperçu des nouvelles features
print("\n📊 Aperçu des features temporelles:")
feature_sample = df_features[['Date', 'Exercice', 'Poids_kg', 'Volume'] + new_features].head(10)
print(feature_sample)

In [ ]:
# Analyse de corrélation des nouvelles features
print("🔍 ANALYSE DE CORRÉLATION DES FEATURES TEMPORELLES")
print("=" * 60)

# Sélection des features numériques pour l'analyse de corrélation
numeric_features = ['Poids_kg', 'Reps', 'Volume', 'Session_Number', 
                   'Volume_Rolling_3', 'Poids_Rolling_3', 'Poids_Progression',
                   'Volume_Progression', 'Pct_Max_Personnel', 'Volume_Cumule']

correlation_matrix = df_features[numeric_features].corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f')
plt.title('🔍 Matrice de Corrélation des Features Temporelles', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Identification des corrélations fortes
print("🔗 Corrélations significatives (|r| > 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]
            print(f"   • {feature1} ↔ {feature2}: {corr_value:.3f}")

## 🎯 Résumé et insights temporels

In [ ]:
print("🎯 RÉSUMÉ ET INSIGHTS TEMPORELS")
print("=" * 60)

# Métriques temporelles globales
total_period = (df['Date'].max() - df['Date'].min()).days
training_days = df['Date'].nunique()
training_frequency = training_days / (total_period + 1) * 7 if total_period > 0 else 0

print(f"📊 MÉTRIQUES TEMPORELLES GLOBALES:")
print(f"   • Période totale: {total_period} jours")
print(f"   • Jours d'entraînement: {training_days}")
print(f"   • Fréquence hebdomadaire: {training_frequency:.1f} jours/semaine")
print(f"   • Régularité: {regularity if 'regularity' in locals() else 'Non calculée'}")

print(f"\n📈 TENDANCES DE PROGRESSION:")
total_exercises_with_progression = len(progression_df[progression_df['Progression_Poids'] > 0]) if 'progression_df' in locals() else 0
total_exercises = len(progression_df) if 'progression_df' in locals() else 0

if total_exercises > 0:
    progression_rate = total_exercises_with_progression / total_exercises * 100
    print(f"   • Exercices en progression: {total_exercises_with_progression}/{total_exercises} ({progression_rate:.1f}%)")
    print(f"   • Progression moyenne: {progression_df['Progression_Poids'].mean():.2f} kg")
    print(f"   • Meilleur exercice: {best_progression['Exercice']} (+{best_progression['Progression_Pct']:.1f}%)")

print(f"\n📅 PATTERNS HEBDOMADAIRES:")
if 'weekly_patterns' in locals() and len(weekly_patterns) > 0:
    best_day = weekly_patterns.loc[weekly_patterns['Volume_Total'].idxmax(), 'Jour_Semaine']
    best_day_volume = weekly_patterns['Volume_Total'].max()
    print(f"   • Jour le plus productif: {best_day} ({best_day_volume:.0f}kg)")
    print(f"   • Répartition équilibrée: {'✅ Oui' if weekly_patterns['Volume_Total'].std() < weekly_patterns['Volume_Total'].mean() * 0.5 else '⚠️ À améliorer'}")

print(f"\n🎲 FEATURES ML GÉNÉRÉES:")
print(f"   • {len(new_features)} nouvelles features temporelles")
print(f"   • Rolling windows pour tendances")
print(f"   • Features cycliques pour saisonnalité")
print(f"   • Métriques de progression individuelles")

print(f"\n🚀 RECOMMANDATIONS TEMPORELLES:")
if training_frequency < 3:
    print(f"   ⚠️ Fréquence d'entraînement faible - augmenter à 3-4 fois/semaine")
elif training_frequency > 6:
    print(f"   ⚠️ Fréquence très élevée - prévoir des jours de repos")
else:
    print(f"   ✅ Fréquence d'entraînement optimale")

if 'regularity' in locals() and '❌' in regularity:
    print(f"   📅 Améliorer la régularité des sessions")
elif 'regularity' in locals() and '🎯' in regularity:
    print(f"   ✅ Excellente régularité maintenue")

if total_exercises > 0 and progression_rate < 50:
    print(f"   📈 Revoir la programmation - moins de 50% des exercices progressent")
elif total_exercises > 0 and progression_rate > 80:
    print(f"   🏆 Excellente progression générale")

print(f"\n📚 PROCHAINES ANALYSES RECOMMANDÉES:")
print(f"   1. Modélisation prédictive avec les features temporelles")
print(f"   2. Détection de plateaux automatisée")
print(f"   3. Recommandations de charge optimale")
print(f"   4. Analyse de saisonnalité sur plus de données")

---
## 📝 Résumé de l'analyse temporelle

Ce notebook a fourni une analyse temporelle complète des données d'entraînement :

### ✅ Analyses réalisées
- **Évolution globale** : Tendances de volume et progression
- **Progression par exercice** : Tracking individuel des performances
- **Patterns hebdomadaires** : Identification des jours optimaux
- **Régularité** : Analyse de la constance d'entraînement
- **Features ML** : 11 nouvelles variables temporelles

### 🎯 Insights clés
- Tendances de progression identifiées
- Patterns comportementaux révélés
- Features prêtes pour modélisation ML
- Recommandations d'optimisation

### 📊 Données générées
- `df_features`: Dataset enrichi avec features temporelles
- `progression_df`: Métriques de progression par exercice
- `weekly_patterns`: Analyse hebdomadaire
- `daily_stats`: Statistiques quotidiennes

**Prochaine étape:** Feature engineering avancé et préparation pour les modèles ML (04_features_engineering.ipynb)